In [1]:
import pickle
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
from matplotlib import pyplot as plt
import math

import dataparsing

device = torch.device("cuda")

# Initialize wandb and dataset

In [2]:
train_dataset = dataparsing.AugmentedDataset(dataparsing.train, transforms=True)
test_dataset = dataparsing.AugmentedDataset(dataparsing.test, transforms=True)
val_dataset = dataparsing.AugmentedDataset(dataparsing.val, transforms=True)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=2)

# Define Vae 

In [3]:
cluster_centers = [0, 0, 0, 0]
def get_corner_vectors(dimension):
    corners = []
    corner1 = torch.ones(dimension)
    corner2 = -torch.ones(dimension)

    corner3 = torch.empty(dimension)
    corner4 = torch.empty(dimension)
    for i in range(dimension):
        if i % 2 == 0:
            corner3[i] = 1
            corner4[i] = -1
        else:
            corner3[i] = -1
            corner4[i] = 1
    scale_factor = 5.0
    corners = [corner1 * scale_factor, corner2 * scale_factor, corner3 * scale_factor, corner4 * scale_factor]

    return [c.to(device) for c in corners]
cluster_centers = get_corner_vectors(128)

In [4]:
class SelfAttention(nn.Module):
    def __init__(self, n_heads, embd_dim, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        self.n_heads = n_heads
        self.in_proj = nn.Linear(embd_dim, 3 * embd_dim, bias=in_proj_bias)
        self.out_proj = nn.Linear(embd_dim, embd_dim, bias=out_proj_bias)
        self.d_heads = embd_dim // n_heads
    def forward(self, x, casual_mask=False):
        batch_size, seq_len, d_embed = x.shape
        interim_shape = (batch_size, seq_len, self.n_heads, self.d_heads)
        q, k, v = self.in_proj(x).chunk(3, dim=-1)
        q = q.view(interim_shape)
        k = k.view(interim_shape)
        v = v.view(interim_shape)
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        weight = q @ k.transpose(-1, -2)
        if casual_mask:
            mask = torch.ones_like(weight, dtype=torch.bool).triu(1)
            weight.masked_fill_(mask, -torch.inf)
        weight /= math.sqrt(self.d_heads)
        weight = F.softmax(weight, dim=-1)
        output = weight @ v
        output = output.transpose(1, 2)
        output = output.reshape((batch_size, seq_len, d_embed))
        output = self.out_proj(output)
        return output

In [5]:
class AttentionBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.groupnorm = nn.GroupNorm(32, channels)
        self.attention = SelfAttention(1, channels)
    def forward(self, x):
        residual = x.clone()
        x = self.groupnorm(x)
        n, c, h, w = x.shape
        x = x.view((n, c, h * w))
        x = x.transpose(-1, -2)
        x = self.attention(x)
        x = x.transpose(-1, -2)
        x = x.view((n, c, h, w))
        x += residual
        return x

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.groupnorm1 = nn.GroupNorm(32, in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.groupnorm2 = nn.GroupNorm(32, out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        
        if in_channels == out_channels:
            self.residual_layer = nn.Identity()
        else:
            self.residual_layer = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
    def forward(self, x):
        residue = x.clone()
        x = self.groupnorm1(x)
        x = F.selu(x)
        x = self.conv1(x)
        x = self.groupnorm2(x)
        x = self.conv2(x)
        return x + self.residual_layer(residue)

In [7]:
class Encoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            ResidualBlock(128, 128),
            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=0),
            ResidualBlock(128, 256),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=0),
            ResidualBlock(256, 512),
            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=0),
            AttentionBlock(512),
            ResidualBlock(512, 512),
            nn.GroupNorm(32, 512),
            nn.SiLU(),
            nn.Conv2d(512, 8, kernel_size=3, padding=1),
            nn.Conv2d(8, 8, kernel_size=1, padding=0)
        )
    def forward(self, x):
        for module in self:
            if isinstance(module, nn.Conv2d) and module.stride == (2, 2):
                x = F.pad(x, (0, 1, 0, 1))
            x = module(x)
        
        return x

In [8]:
class Decoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Conv2d(4, 512, kernel_size=3, padding=1),
            ResidualBlock(512, 512),
            AttentionBlock(512),
            ResidualBlock(512, 512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            ResidualBlock(512, 512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            ResidualBlock(512, 256),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            ResidualBlock(256, 128),
            nn.GroupNorm(32, 128),
            nn.SiLU(),
            nn.Conv2d(128, 3, kernel_size=3, padding=1)
        )
    def forward(self, x):
        x /= 0.18215
        for module in self:
            x = module(x)
        return x

In [9]:
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.mu = nn.Linear(512, 128)
        self.logvar = nn.Linear(512, 128)
        self.restarter = nn.Linear(128, 256)
    def forward(self, x):
        encoded = self.encoder(x)
        encoded = encoded.view(encoded.size(0), -1)
        mean = self.mu(encoded)
        log_variance = self.logvar(encoded)
        log_variance = torch.clamp(log_variance, -30, 20)
        std = torch.exp(0.5 * log_variance)
        eps = torch.randn_like(std)
        mean = mean + eps * std
        restarted = self.restarter(mean)
        restarted = restarted.view(restarted.size(0), 4, 8, 8)
        decoded = self.decoder(restarted)
        return decoded, mean, log_variance


In [10]:
vae = VAE()
vae.to(device)
optimizer = torch.optim.AdamW(vae.parameters(), lr=1e-3)

# Define Loss

In [11]:
def loss_function(mu, y, recon_x, x):
    # mu: [batch_size, latent_dim], y: [batch_size] with values in [0..L-1]
    centers = torch.stack(cluster_centers).to(mu.device)        # [L, latent_dim]
    distances = torch.cdist(mu, centers)                       # [batch_size, L]

    # invert distances to get logits; add eps to avoid div0
    total_dist = distances.sum(dim=1, keepdim=True)            # [batch_size, 1]
    logits = (total_dist - distances) / (total_dist + 1e-8)    # [batch_size, L]

    loss = F.cross_entropy(logits, y)
    preds = logits.argmax(dim=1)
    acc  = (preds == y).float().mean()
    mse_loss = F.mse_loss(recon_x, x, reduction='sum')
    loss += mse_loss * 0.00001
    return loss, acc

# Function to sample anchor, positive, negative triplets within a batch
def sample_triplets(mu, labels):
    anchors, positives, negatives = [], [], []
    device = mu.device  # Get the device of the mu tensor
    for i in range(mu.size(0)):
        anchor, label = mu[i], labels[i]
        # Ensure the index tensor is on the same device as mu
        positive_indices = (labels == label) & (torch.arange(mu.size(0), device=device) != i)
        negative_indices = labels != label
        positive = mu[positive_indices]
        negative = mu[negative_indices]

        if len(positive) > 0 and len(negative) > 0:
            pos_sample = positive[torch.randint(len(positive), (1,), device=device)]
            neg_sample = negative[torch.randint(len(negative), (1,), device=device)]
            anchors.append(anchor)
            positives.append(pos_sample.squeeze(0))
            negatives.append(neg_sample.squeeze(0))
    if len(anchors) > 0:
        return torch.stack(anchors), torch.stack(positives), torch.stack(negatives)
    else:
        return None, None, None

# Training Loop

In [12]:
for epoch in range(40):
    vae.train()
    running_loss = 0.0
    running_acc = 0.0
    total_samples = 0


    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        inputs = batch[0].to(device)
        labels = batch[1].to(device) # Get labels for loss calculation
        optimizer.zero_grad()
        recon_x, mu, logvar = vae(inputs)
        # Use the fixed cluster_centers in the loss function
        loss, accuracy = loss_function(mu, labels, recon_x, inputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy * inputs.size(0)
        total_samples += inputs.size(0)


    avg_train_loss = running_loss / total_samples

    # Validation loop
    vae.eval()
    running_val_loss = 0.0
    running_acc_eval = 0.0
    total_val_samples = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} - Validation"):
            inputs = batch[0].to(device)
            labels = batch[1].to(device) # Get labels for loss calculation
            recon_x, mu, logvar = vae(inputs)
            # Use the fixed cluster_centers in the loss function
            loss, acc = loss_function(mu, labels, recon_x, inputs)
            running_val_loss += loss.item() * inputs.size(0)
            running_acc_eval += acc * inputs.size(0)
            total_val_samples += inputs.size(0)

    avg_val_loss = running_val_loss / total_val_samples


    # Save a sample input and its reconstruction as an image
    vae.eval()
    sample_batch = next(iter(val_loader))
    sample_input = sample_batch[0].to(device)
    with torch.no_grad():
        sample_output, _, _ = vae(sample_input)
    sample_input = sample_input.cpu().numpy()
    sample_output = sample_output.cpu().numpy()

    num_samples = min(20, sample_input.shape[0])
    fig, axs = plt.subplots(2, num_samples, figsize=(num_samples * 2, 4))
    for i in range(num_samples):
        # Input image
        axs[0, i].imshow(sample_input[i].transpose(1, 2, 0))
        axs[0, i].axis("off")
        # Reconstructed image
        axs[1, i].imshow(sample_output[i].transpose(1, 2, 0))
        axs[1, i].axis("off")
    plt.savefig(f"vae_reconstructions/epoch_{epoch+1}.png")
    plt.close(fig)

    print(f"Train Loss: {avg_train_loss}, Train Acc: {running_acc / total_samples}, Val Loss: {avg_val_loss}, Val Acc: {running_acc_eval / total_val_samples}")

    torch.save(vae, 'VAEModelCubicFit.pth')

Epoch 1 - Validation: 100%|██████████| 395/395 [00:34<00:00, 11.61it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.019793559..1.0267051].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.01352371..0.9410879].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.015502038..0.89859116].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.019507084..0.95978993].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.009365729..0.88491064].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0050186887..0.957394].
Clipping input data to the valid range for i

Train Loss: 1.3718013155009738, Train Acc: 0.5398625135421753, Val Loss: 1.3673725184472352, Val Acc: 0.5353407263755798


Epoch 2 - Validation: 100%|██████████| 395/395 [00:34<00:00, 11.52it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.012777794..0.9806065].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.014203019..0.5395645].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0038332548..1.0283058].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.006150985..0.882353].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.006701395..1.007824].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.014931789..1.0070276].
Clipping input data to the valid range for imshow 

Train Loss: 1.350092654703409, Train Acc: 0.581134021282196, Val Loss: 1.3501503845024412, Val Acc: 0.5793977975845337


Epoch 3 - Validation: 100%|██████████| 395/395 [00:34<00:00, 11.49it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.013369761..0.7311668].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0053419396..0.9263864].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.021733388..0.7800881].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0031787157..0.86840236].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.027555563..1.0931821].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.013667569..0.84882957].


...

Train Loss: 1.28415377017149, Train Acc: 0.7146391868591309, Val Loss: 1.2865140529895547, Val Acc: 0.7099841833114624


Epoch 30 - Validation: 100%|██████████| 395/395 [00:34<00:00, 11.60it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.03312549..0.9654236].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0036900267..1.0147707].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.010211468..1.0180722].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.010369092..0.93442184].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0061763376..0.6185344].


Train Loss: 1.2818181074607824, Train Acc: 0.722542941570282, Val Loss: 1.2880555475948352, Val Acc: 0.7028526067733765


Epoch 31 - Training:  62%|██████▏   | 1125/1819 [03:21<02:04,  5.58it/s]


KeyboardInterrupt: 